<a href="https://colab.research.google.com/github/wldud01/Pytorch_tutorial/blob/main/Torch_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
### A Gentle Introduction to torch.autograd
torch.autograd is pytorch automatic differenctiation engine that powers neural network training.

- Background
NNs fare collection of nested functions that aree executed on some input data. These functions are defined by parameters, which in Pytorch are stored in tensor.
  - Trainig a NN happens in two steps

  Forward Propagation: make its best guess about the correct output. It runs the input data through each of its functions to make this guess.

  Backward Propagation: NNs adjusts its parameters proportionate to the error in its guess. It des this by traversing backwards of the functions(gradients), and optimizing the parameters using gradient descent.

In [9]:
#@title Usage in pytorch
# load a pretrained model resnet 18 model from torchvision
# create a random data tensor to represent a single image with 3 channels
# Label in pretrained models has shape(1,1000)
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
# 1 image 3 chanel size 64
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)
# RAM 1.4GB

In [10]:
print(data.device)

cpu


In [11]:
#@title prediction
# input data through the model - forward pass
prediction =  model(data)

In [12]:
#@title loss
# caculate the error
# to backpropagate this error through the network
# kicked off when we call .backwards() on the error tensor
# Autograd calculates and stores the gradients for each model parameter in the parameters .grad attribute
loss = (prediction - labels).sum()
loss.backward() # backward pass

In [14]:
# each model paramter in the parameter's .grad attribute
loss.grad

<ipython-input-14-09a6243c8d35>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  loss.grad


In [15]:
#@title optimization
# we register all the parameters of the model in the optimizer
optim = torch.optim.SGD(model.parameters(),lr=1e-2, momentum =0.9)

In [16]:
optim.step() # gradient descent
# optimizer adjusts each parameters by its gradient stored in . grad

In [17]:
#@title Differentiation in Autograd
# take a look at how autograd collects gradients
# tensor a, b  required grad = True ,
# this signal to autograd that every operation on them should be tracked

import torch
# Input vector
a = torch.tensor([2.,3.],requires_grad = True)
b = torch.tensor([6.,4.], requires_grad=True)

In [18]:
# create another tensor Q
# like loss function
Q = 3*a**3-b**2

In [20]:
# a,b is parameters and Q is to be the error
# when we call .backward() on Q, autograd calculates these gradients and stores the,
# gradient is a tensor of the same shape as Q, and it represents the gradient of Q itself
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

In [21]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


@title vector caculus using autograd
jacobian product가 each parameters differenciate
Generally speaking torch.autograd isf an engine for computing vector-Jacobian product

The backward pass kicks off when .backward() is called on the DAG root.
- Compute the gradients from each .grad_fn
- accumulates them in the respective tensor's .grad attribute and
- using the chain rule, propagatess all the way to the leaf tensors



In [3]:
#@title Exclusion from the DAG
import torch

x = torch.rand(5,5)
y = torch.rand(5,5)
z = torch.rand((5,5),requires_grad=True)

a = x+y
# frozen parameters
print(a.requires_grad)
b = x +z
print(b.requires_grad)

False
True


In [5]:
# In finetuning , we greeze most of the model
# and only modify the classifier layers to make predictions

from torch import nn, optim
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# freeze all parameters
for param in model.parameters():
  param.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 121MB/s]


In [6]:
# fine tuning the model on a anew dataset with 10 labels
# model.fc is the las linear layer
# this code can simply replace it with a new linear layer
model.fc = nn.Linear(512,10)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-2, momentum = 0.9)